In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#list the batch folder where all pdfs are 
#the folder

train_batches="/gpfsscratch/rech/zpf/uyf36me/training_batches"

import os

order=['batch_28',
 'batch_18',
 'batch_7',
 'batch_2',
 'batch_0',
 'batch_108',
 'batch_106',
 'batch_85',
 'batch_123',
 'batch_48',
 'batch_114',
 'batch_125',
 'batch_67',
 'batch_4',
 'batch_93',
 'batch_3',
 'batch_86',
 'batch_36',
 'batch_41',
 'batch_100']

batches=[os.path.join(train_batches,batch) for batch in order]


#reordering file name accroding to some predefined run (which is here nlp)
train_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/**.csv"
import glob
files=glob.glob(train_path)

file_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/roberta_from_scratch_ft.txt"

import os
base="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/"
with open(file_path,"r") as f:
    lines=f.readlines()
    
start=[]
for line in lines:
    val=line.split(",")[0]
    start.append(val)

start=files
start
# print(start)
        
batches,csv_files=batches,files

#align csvs
csv_order=[]
for batch in batches:
    
    batch_name=batch.rsplit("/")[-1]

    
    for element in csv_files:
        if(batch_name+".csv" in element):
            
            csv_order.append(element)

In [ ]:
#start to build image dataset from this csv order using the batches
main_folder=batches[2]
csv_file=csv_order[2]
print(main_folder,csv_file)

In [ ]:
#make sql connection to the table
#apply filteration steps
#then fetch one image and display
import math
import random
import warnings
import sqlite3
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd


val_table=pd.read_csv(csv_file)

print(val_table.shape)

#freeze the base model
def setting_up_labels(label):
    label=label.lower()
    if("basic" in label):
        return "basic"
    if("overlap" in label):
        return "overlap"
    if("proof" in label):
        return "proof"
    if("theorem" in label):
        return  "theorem"
    else:
        return "theorem"
    
    
def setting_up_labels_1(label):
    label=label.lower()
    if("basic" in label):
        return 0
    if("overlap" in label):
        return 3
    if("proof" in label):
        return 1
    if("theorem" in label):
        return  2
    
val_table["label"]=val_table["label"].apply(setting_up_labels)
val_table=val_table.dropna()
val_table["label"]=val_table["label"].apply(setting_up_labels_1)



In [ ]:
pdfs=list(val_table["pdf_path"].unique())
print(len(pdfs))

In [ ]:
#size

In [ ]:
val_table

In [ ]:
print(val_table["page_no"].iloc[0])

In [ ]:
from ast import literal_eval

val_table['page_no'] = val_table["page_no"].apply(lambda x: literal_eval(str(x)))
val_table['top_left'] = val_table["top_left"].apply(lambda x: literal_eval(str(x)))
val_table['bot_right'] = val_table["bot_right"].apply(lambda x: literal_eval(str(x)))


In [ ]:
#print(train_table.shape) #has to be 6 columned

print(val_table.shape)

In [ ]:
import pandas as pd
from ast import literal_eval

def make_csv_for_extracting_images(csv_path):
    val_table=pd.read_csv(csv_path)

    #freeze the base model
    def setting_up_labels(label):
        label=label.lower()
        if("basic" in label):
            return "basic"
        if("overlap" in label):
            return "overlap"
        if("proof" in label):
            return "proof"
        if("theorem" in label):
            return  "theorem"
        else:
            return "theorem"


    def setting_up_labels_1(label):
        label=label.lower()
        if("basic" in label):
            return 0
        if("overlap" in label):
            return 3
        if("proof" in label):
            return 1
        if("theorem" in label):
            return  2

    val_table["label"]=val_table["label"].apply(setting_up_labels)
    val_table=val_table.dropna()
    val_table["label"]=val_table["label"].apply(setting_up_labels_1)
    

    val_table['page_no'] = val_table["page_no"].apply(lambda x: literal_eval(str(x)))
    val_table['top_left'] = val_table["top_left"].apply(lambda x: literal_eval(str(x)))
    val_table['bot_right'] = val_table["bot_right"].apply(lambda x: literal_eval(str(x)))

    
    return val_table

In [13]:
def extract_and_save_image(row,folder_name,image_location,ext=".png"):
    try:
        label=row[1]["label"]
        local_folder=image_location+"/"+folder_name+"/label_"+str(label)
        if(not os.path.exists(local_folder)):
            os.mkdir(local_folder)
        image_path=row[1]["pdf_path"].rsplit("/",1)[0]
        sub_folder=image_path.split(".")[0]
        generic_path=image_path+"/images"+"/image_{}.png".format(str(row[1]["page_no"]))
        file_name=generic_path
        full_path=os.path.join(results_directory,file_name)
        img_file_name=str(generic_path.rsplit("/",2)[0])+"_"+str(row[1]["page_no"])+"_"+str(row[0])+ext
        full_name=os.path.join(local_folder,img_file_name)
        if(os.path.exists(full_name)):
            return 1
        img = cv2.imread(full_path) #add 0 for grey scale
        top_left=math.floor(row[1]["top_left"][0]),math.floor(row[1]["top_left"][1])
        bot_right=math.floor(row[1]["bot_right"][0]),math.floor(row[1]["bot_right"][1])
        cropped_image = img[top_left[1]:bot_right[1], top_left[0]:bot_right[0]]
        status=cv2.imwrite(full_name, cropped_image)
    except TypeError or FileExistsError:
        return
    except:
        print("error--",row[1]["pdf_path"])
        return


In [15]:
for ind in range(0,100):
    batch,csv=batches[ind],csv_order[ind]
    print(batch)
    print(csv)

    results_directory=batch
    df=make_csv_for_extracting_images(csv)

    pdfs_in_csv=[pdf.split("/")[0] for pdf in df["pdf_path"].unique()]
    pdfs_in_folder=os.listdir(batch)
    print(pdfs_in_csv[0],pdfs_in_folder[0])

    cnt=0
    for pdf in pdfs_in_csv:
        if(pdf in pdfs_in_folder):
            cnt+=1
    print(cnt)


/gpfsscratch/rech/zpf/uyf36me/training_batches/batch_28
/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_28.csv


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [16]:
pdfs=[]

for pdf in df["pdf_path"].unique():
    pdfs.append(pdf.split("/")[0])
    
folders=os.listdir("/gpfsscratch/rech/zpf/uyf36me/training_batches/batch_7")

cnt=0
for pdf in pdfs:
    if(pdf not in folders):
        cnt+=1

NameError: name 'df' is not defined

In [32]:
pdfs[:5]

['1106.2378', '1611.08464', '1104.4879', '1511.07907', '1409.6086']

In [14]:
image_location="/gpfsscratch/rech/zpf/uyf36me/training_patches" #path to the dir where you will store the patches this should exist before
folder_name="batch_7"
results_directory="/gpfsscratch/rech/zpf/uyf36me/training_batches/batch_7" #the directory where you saved data csvs images etc
import time
from tqdm import tqdm
tick=time.time()

val_table=make_csv_for_extracting_images(csv_order[2])
for row in tqdm(val_table.iterrows()):
    if(not os.path.exists(os.path.join(image_location,folder_name))):
        os.mkdir(os.path.join(image_location,folder_name))
    df=val_table
    extract_and_save_image(row,folder_name,image_location,ext=".png")
    break
    
tock=time.time()
print(tock-tick)

0it [00:00, ?it/s]

10.877717733383179


In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm

image_location="/gpfsscratch/rech/zpf/uyf36me/training_patches"
results_directory="/gpfsscratch/rech/zpf/uyf36me/training_batches/batch_7" #the directory where you saved data csvs images etc
n_jobs=-2

print(csv_order[2])
df=make_csv_for_extracting_images(csv_order[2])
print(df.shape)

for df,folder_name in zip([df],["batch_7"]):
    if(not os.path.exists(os.path.join(image_location,folder_name))):
        os.mkdir(os.path.join(image_location,folder_name))
    res=Parallel(n_jobs=n_jobs,backend="threading",verbose=2)(delayed(extract_and_save_image)
                                           (row,folder_name,image_location) for row in tqdm(df.iterrows()))

/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_7.csv
(201518, 26)


0it [00:00, ?it/s][Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 39 concurrent workers.
1it [00:00,  4.36it/s][Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-2)]: Done 287 tasks      | elapsed:    0.3s
429it [00:00, 1618.89it/s][Parallel(n_jobs=-2)]: Done 570 tasks      | elapsed:    0.4s
819it [00:00, 2413.06it/s][Parallel(n_jobs=-2)]: Done 935 tasks      | elapsed:    0.5s
1209it [00:00, 2875.49it/s][Parallel(n_jobs=-2)]: Done 1380 tasks      | elapsed:    0.6s
1875it [00:00, 3054.23it/s][Parallel(n_jobs=-2)]: Done 1907 tasks      | elapsed:    0.8s
2548it [00:00, 3054.25it/s][Parallel(n_jobs=-2)]: Done 2514 tasks      | elapsed:    1.0s
3167it [00:01, 2622.55it/s][Parallel(n_jobs=-2)]: Done 3203 tasks      | elapsed:    1.3s
3744it [00:01, 2592.59it/s][Parallel(n_jobs=-2)]: Done 3972 tasks      | elapsed:    1.6s
4676it [00:01, 2540.61it/s][Parallel(n_jobs=-2)]: Done 4823 tasks      | elapsed:    1.9s
5609it [00:02, 2933.49it/s][Parallel(n

In [40]:
results_directory="/gpfsscratch/rech/zpf/uyf36me/training_patches/batch_2/**/**.png"
files=glob.glob(results_directory)
unique=set([file.split("/")[-1].split("_")[0] for file in files])
#print(unique)
print(len(unique))

0


In [15]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm

image_location="/gpfsscratch/rech/zpf/uyf36me/training_patches"

n_jobs=-2
ind=10

batch,csv=batches[ind],csv_order[ind]
results_directory=batch
df=make_csv_for_extracting_images(csv)
print(df.shape)
folder_name=batch.split("/")[-1]
print("----doing batch", folder_name)
if(not os.path.exists(os.path.join(image_location,folder_name))):
    os.mkdir(os.path.join(image_location,folder_name))
res=Parallel(n_jobs=n_jobs,backend="threading",verbose=2)(delayed(extract_and_save_image)
                                       (row,folder_name,image_location) for row in tqdm(df.iterrows()))



(199146, 26)
----doing batch batch_114


0it [00:00, ?it/s][Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 39 concurrent workers.
178it [00:10,  4.99it/s][Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   10.4s
351it [00:21, 28.73it/s][Parallel(n_jobs=-2)]: Done 287 tasks      | elapsed:   21.5s
624it [00:39, 28.04it/s][Parallel(n_jobs=-2)]: Done 570 tasks      | elapsed:   39.4s
984it [01:03,  8.31it/s][Parallel(n_jobs=-2)]: Done 935 tasks      | elapsed:  1.1min
1421it [01:32, 11.91it/s][Parallel(n_jobs=-2)]: Done 1380 tasks      | elapsed:  1.5min
1958it [02:07, 12.99it/s][Parallel(n_jobs=-2)]: Done 1907 tasks      | elapsed:  2.1min
2338it [02:31, 27.66it/s]Exception in thread Thread-47 (_handle_results):
Traceback (most recent call last):
  File "/gpfslocalsup/pub/anaconda-py3/2021.05/envs/tensorflow-2.9.1+py3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/gpfslocalsup/pub/anaconda-py3/2021.05/envs/tensorflow-2.9.1+py3.10/lib/python3.10/threading.py", line 953, in 

In [17]:
from ast import literal_eval
#freeze the base model
def setting_up_labels(label):
    label=label.lower()
    if("basic" in label):
        return "basic"
    if("overlap" in label):
        return "overlap"
    if("proof" in label):
        return "proof"
    if("theorem" in label):
        return  "theorem"
    
def setting_up_labels_1(label):
    label=label.lower()
    if("basic" in label):
        return 0
    if("overlap" in label):
        return 3
    if("proof" in label):
        return 1
    if("theorem" in label):
        return  2
    

def preprocess_df_for_vision(df):
    #string conversion
    df['page_no'] = df["page_no"].apply(lambda x: literal_eval(str(x)))
    df['top_left'] = df["top_left"].apply(lambda x: literal_eval(str(x)))
    df['bot_right'] = df["bot_right"].apply(lambda x: literal_eval(str(x)))
    df["label"]=df["label"].apply(setting_up_labels)
    #dropping nans same as in NLP approach
    df=df.dropna()
    df["label"]=df["label"].apply(setting_up_labels_1)
    
    return df

#order of execution
names=['batch_28.csv',
 'batch_18.csv',
 'batch_108.csv',
 'batch_106.csv',
 'batch_85.csv',
 'batch_123.csv',
 'batch_7.csv',
 'batch_48.csv',
 'batch_2.csv',
 'batch_114.csv',
 'batch_125.csv',
 'batch_67.csv',
 'batch_4.csv']

In [61]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm
import glob

image_location="/Users/mv96/Downloads/vision_patches"
results_directory="/Volumes/My_Book/Theoremkb/cluster_package/5-grobid_on_pdfs/ftest" #the directory where you saved data csvs images etc
n_jobs=-2

train_data_dir="/Volumes/My_Book/Theoremkb/cluster_package/8-finetuning_data/train_data"
csv_files=glob.glob(train_data_dir+"/**.csv")

order=[]
for name in names:
    for file in csv_files:
        if(name in file):
            order.append(file)

In [ ]:
for file in tqdm(order):
    folder_name=file[:-4].rsplit("/")[-1]
    df=pd.read_csv(file)
    df=preprocess_df_for_vision(df)
    print(folder_name,df.shape)
    
    #continue if the zip file or folder they exist
    zip_path=os.path.join(image_location,folder_name)+".zip"
    if(os.path.exists(zip_path)):
        continue
    if(not os.path.exists(os.path.join(image_location,folder_name))):
        os.mkdir(os.path.join(image_location,folder_name))
    else:
        continue
    res=Parallel(n_jobs=n_jobs,backend="threading",verbose=2)(delayed(extract_and_save_image)
                                           (row,df,folder_name,image_location) for row in tqdm(df.iterrows()))
    


In [4]:
import os
image_location="/Volumes/MV96/vision_patches"
check_dir=os.path.join(image_location,"test_data")
print(check_dir)
counts=0
for label in os.listdir(check_dir):
    counts+=len(os.listdir(os.path.join(check_dir,label)))
    #print(train_table.shape)
print(counts)    

/Volumes/MV96/vision_patches/test_data
76986


17511